In [12]:
!pip install tensorflow==2.2.0

  Using cached tensorflow-2.2.0-cp37-cp37m-win_amd64.whl (459.2 MB)
  Using cached tensorflow_estimator-2.2.0-py2.py3-none-any.whl (454 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\divya\\anaconda3\\envs\\divenv\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# import keras
# from tensorflow import keras
from tensorflow.keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Input

In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:XLA_GPU:0']


In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
! conda info -e

# conda environments:
#
base                     C:\Users\divya\anaconda3
divenv                *  C:\Users\divya\anaconda3\envs\divenv
gpu_test                 C:\Users\divya\anaconda3\envs\gpu_test
gpu_test_1               C:\Users\divya\anaconda3\envs\gpu_test_1



In [5]:
tf.__version__

'2.2.0'

In [6]:
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [7]:
# import zipfile
# path = r'C:\Users\divya\PycharmProjects\Applied_ai\transfer_learning\aaic-assignment-tl.zip'
# unzip_path = r'C:\Users\divya\PycharmProjects\Applied_ai\transfer_learning\aaic-assignment-tl'
# with zipfile.ZipFile(path, 'r') as zip_ref:
#     zip_ref.extractall(unzip_path)

In [8]:
import os
os.getcwd()

'C:\\Users\\divya\\PycharmProjects\\Applied_ai\\transfer_learning'

In [2]:
df = pd.read_csv(r'labels_final.csv',dtype=str)

In [3]:
df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [4]:
data_gen = ImageDataGenerator(rescale=1/255., validation_split=0.25)

In [5]:
colnames=['path', 'label'] 
train_data = data_gen.flow_from_dataframe(dataframe=df, directory="./aaic-assignment-tl/data_final", x_col=colnames[0], y_col=colnames[1], 
                                          class_mode="categorical", target_size=(256,256), batch_size=32, subset='training', seed = 9)

Found 36000 validated image filenames belonging to 16 classes.


In [6]:
validation_data = data_gen.flow_from_dataframe(dataframe=df, directory="./aaic-assignment-tl/data_final", x_col=colnames[0], y_col=colnames[1], 
                                          class_mode="categorical", target_size=(256,256), batch_size=32, subset='validation', seed = 9)

Found 12000 validated image filenames belonging to 16 classes.


In [7]:
import tensorflow as tf
# import keras
from tensorflow.keras.applications.vgg16 import VGG16
def model_1(input_shape = (256,256,3), n_classes = 16):
  # X_input = Input(input_shape)
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape)
    for layer in vgg.layers:
        layer.trainable = False
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=8,kernel_size=(3,3),padding="same",strides=2, activation="relu")(vgg_out)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(32, activation = 'relu')(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model


In [16]:
model_1 = model_1(input_shape = (256,256,3), n_classes = 16)
print(model_1.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [17]:
model_1.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])

In [18]:
batch_size = 32
train_steps = train_data.samples // batch_size
val_steps = validation_data.samples // batch_size

In [ ]:
history = model_1.fit(train_data,steps_per_epoch=train_steps, epochs=20,
                              validation_data=validation_data,validation_steps=val_steps)

Epoch 1/20
1125/1125 [==============================] - 716s 637ms/step - loss: 1.6358 - accuracy: 0.4869 - val_loss: 1.3520 - val_accuracy: 0.5860
Epoch 2/20
1125/1125 [==============================] - 678s 602ms/step - loss: 1.2390 - accuracy: 0.6182 - val_loss: 1.2294 - val_accuracy: 0.6248
Epoch 3/20
1125/1125 [==============================] - 555s 493ms/step - loss: 1.1168 - accuracy: 0.6562 - val_loss: 1.1428 - val_accuracy: 0.6546
Epoch 4/20
1125/1125 [==============================] - 556s 494ms/step - loss: 1.0344 - accuracy: 0.6812 - val_loss: 1.1269 - val_accuracy: 0.6646
Epoch 5/20
1125/1125 [==============================] - 553s 492ms/step - loss: 0.9748 - accuracy: 0.6998 - val_loss: 1.1122 - val_accuracy: 0.6690
Epoch 6/20
1125/1125 [==============================] - 545s 484ms/step - loss: 0.9265 - accuracy: 0.7160 - val_loss: 1.1220 - val_accuracy: 0.6728
Epoch 7/20
1125/1125 [==============================] - 539s 479ms/step - loss: 0.8836 - accuracy: 0.7275 - val_

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 1")
plt.show()

In [ ]:
tf.keras.utils.plot_model(model_1, to_file='model_1.png')

In [ ]:
def model_2(input_shape = (256,256,3), n_classes = 16):
  # X_input = Input(input_shape)
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape)
    for layer in vgg.layers:
        layer.trainable = False
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=4096,kernel_size=8,padding="same",strides=1, activation="relu")(vgg_out)
    x = Conv2D(filters=4096,kernel_size=1,padding="same",strides=1, activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model

In [ ]:
model_2 = model_2(input_shape = (256,256,3), n_classes = 16)
print(model_2.summary())

In [ ]:
model_2.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])
history = model_2.fit(train_data,steps_per_epoch=train_steps, epochs=20,
                              validation_data=validation_data,validation_steps=val_steps)

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 1")
plt.show()

In [ ]:
tf.keras.utils.plot_model(model_2, to_file='model_2.png')

In [8]:
def model_3(input_shape = (256,256,3), n_classes = 16):
  # X_input = Input(input_shape)
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape)
    for layer in vgg.layers[-6:]:
        layer.trainable = True
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=4096,kernel_size=8,padding="same",strides=1, activation="relu")(vgg_out)
    x = Conv2D(filters=4096,kernel_size=1,padding="same",strides=1, activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model

In [ ]:
model_3 = model_3(input_shape = (256,256,3), n_classes = 16)
print(model_3.summary())

In [ ]:
model_3.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])
history = model_3.fit(train_data,steps_per_epoch=train_steps, epochs=20,
                              validation_data=validation_data,validation_steps=val_steps)

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 1")
plt.show()

In [ ]:
tf.keras.utils.plot_model(model_3, to_file='model_3.png')